# Dataset: OutreachData.csv
# Data Cleaning Process

Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd

Loading Dataset 

In [2]:
df = pd.read_csv("OutreachData.csv")

Total Rows & Columns

In [3]:
print(f"Total Rows: {df.shape[0]}")
print(f"Total Columns: {df.shape[1]}")

Total Rows: 37881
Total Columns: 8


Missing Values Checking

In [4]:
print(df.isnull().sum())

Reference_ID               0
Received_At                0
University                 0
Caller_Name                0
Outcome_1                  0
Remark                 33804
Campaign_ID                0
Escalation_Required        0
dtype: int64


Replacing missing value with "No Remark" Value in Remark Column

In [5]:
df['Remark'] = df['Remark'].fillna('No Remark')

In [6]:
df['Remark'].head(10)

0          No Remark
1          No Remark
2          No Remark
3          No Remark
4          No Remark
5          No Remark
6          No Remark
7    within few days
8          No Remark
9          No Remark
Name: Remark, dtype: object

Duplicate Rows Detecting

In [7]:
duplicates = df.duplicated().sum()
print("Duplicate Rows : ",duplicates)

Duplicate Rows :  0


Checking Datatype

In [8]:
print(df.dtypes)

Reference_ID           object
Received_At            object
University             object
Caller_Name            object
Outcome_1              object
Remark                 object
Campaign_ID            object
Escalation_Required    object
dtype: object


correcting datatype

In [9]:
# Converting Received_At to datetime only
df['Received_At'] = pd.to_datetime(df['Received_At'], format='%m-%d-%Y %H:%M:%S', errors='raise')

df['Reference_ID'] = df['Reference_ID'].astype(str)

# Keeping all other columns as object (no category conversion because power BI can't recognize category but object)
# Check datatypes
print(df.dtypes)

Reference_ID                   object
Received_At            datetime64[ns]
University                     object
Caller_Name                    object
Outcome_1                      object
Remark                         object
Campaign_ID                    object
Escalation_Required            object
dtype: object


Checking Received_At column if it looks fine or not

In [10]:
df['Received_At'].head(10)

0   2023-04-28 12:15:19
1   2023-04-28 13:04:05
2   2023-05-01 11:14:11
3   2023-05-01 11:16:09
4   2023-05-01 11:18:02
5   2023-05-01 11:19:05
6   2023-05-01 11:21:43
7   2023-05-01 11:26:12
8   2023-05-01 11:29:06
9   2023-05-01 11:30:00
Name: Received_At, dtype: datetime64[ns]

Checking Inconsistences

In [11]:
# The list of all object-type columns
object_cols = df.select_dtypes(include='object').columns

# checking unique values for each object column
for col in object_cols:
    print(f"\nUnique values in '{col}':")
    print(df[col].unique())


Unique values in 'Reference_ID':
['12345' '347397' '358065' ... '.....' ',,,,,' '/////////']

Unique values in 'University':
['Illinois Institute of Technology']

Unique values in 'Caller_Name':
['Shailja' 'Isha' 'Poppy' 'Namrata' 'Palak' 'Mounika' 'Twinkle' 'Rudra'
 'Pranjal' 'Prajwal' 'Shrutish' 'Jyoti']

Unique values in 'Outcome_1':
['Connected' 'Reschedule' 'Not connected' 'Will Submit the docx'
 'Completed application' 'Disconnected' 'Voicemail' 'Not interested'
 'Want to defer' 'Wrong number' 'Not interested to IIT'
 'Ready to pay the deposit' 'Not interested to Pay' 'Will confirm later'
 'Already paid the deposit' 'Duplicate app' 'Still making a decision'
 'Looking to defer admission to a future term (SP25 or FA25)'
 'Will work on providing documents soon, still interested in FA24'
 'Student is looking to defer to the SP25 or FA25 term'
 'Student has the needed information, does not need assistance, and plans to enroll soon'
 'Student is having trouble contacting their academi

Reference_ID

Some strange entries: ',,,,,', '/////////'

Action: Keep only numeric/valid IDs.

In [12]:
# Create a mask for invalid Reference_IDs (non-numeric)
mask_invalid = ~df['Reference_ID'].str.isnumeric()

# Count how many rows will be removed
removed_count = mask_invalid.sum()
print("Total rows removed due to invalid Reference_IDs:", removed_count)

# Keep only rows with valid Reference_IDs
df = df[df['Reference_ID'].str.isnumeric()]


Total rows removed due to invalid Reference_IDs: 4762


Removing rows which Reference_ID = 0 (Reference_ID of 0 is usually invalid — IDs typically start from 1 or higher)

In [13]:
# Remove invalid Reference_IDs (non-numeric or 0)
mask_invalid = ~df['Reference_ID'].str.isnumeric() | (df['Reference_ID'] == '0')

# Count how many rows will be removed
removed_count = mask_invalid.sum()
print("Total rows removed due to invalid or 0 Reference_IDs:", removed_count)

# Keep only valid Reference_IDs
df = df[~mask_invalid]


Total rows removed due to invalid or 0 Reference_IDs: 1


University

Only one value → fine, no action needed.

Caller_Name

Looks clean → just strip spaces to be safe.

In [14]:
df['Caller_Name'] = df['Caller_Name'].str.strip()

Outcome_1

Very long text for some outcomes → Power BI can handle it, but might want standardization:

Removing leading/trailing spaces

Fixig inconsistent casing

In [15]:
df['Outcome_1'] = df['Outcome_1'].str.strip()

Remark

Mostly clean, already filled 'No Remark'

Strip spaces for safety

In [16]:
df['Remark'] = df['Remark'].str.strip()

Campaign_ID

Looks fine, strip spaces

In [17]:
df['Campaign_ID'] = df['Campaign_ID'].str.strip()

Escalation_Required

Values: 'No', 'Yes', 'Yes, No' → inconsistent

Action: Standardizing of 'Yes,No' → 'Yes'

Reason: we converting 'Yes, No' to 'Yes' because For dashboards → any instance of escalation is important to track.

In [18]:
df['Escalation_Required'] = df['Escalation_Required'].str.strip()
df['Escalation_Required'] = df['Escalation_Required'].replace({'Yes, No': 'Yes'})

Checking Consistency

In [19]:
# List all object-type columns
object_cols = df.select_dtypes(include='object').columns

# unique values for each object column
for col in object_cols:
    print(f"\nUnique values in '{col}':")
    print(df[col].unique())


Unique values in 'Reference_ID':
['12345' '347397' '358065' ... '98077675' '9876789' '425358']

Unique values in 'University':
['Illinois Institute of Technology']

Unique values in 'Caller_Name':
['Shailja' 'Isha' 'Poppy' 'Namrata' 'Palak' 'Mounika' 'Twinkle' 'Rudra'
 'Pranjal' 'Prajwal' 'Shrutish' 'Jyoti']

Unique values in 'Outcome_1':
['Connected' 'Reschedule' 'Not connected' 'Will Submit the docx'
 'Completed application' 'Disconnected' 'Voicemail' 'Not interested'
 'Want to defer' 'Wrong number' 'Not interested to IIT'
 'Ready to pay the deposit' 'Not interested to Pay' 'Will confirm later'
 'Already paid the deposit' 'Duplicate app' 'Still making a decision'
 'Looking to defer admission to a future term (SP25 or FA25)'
 'Will work on providing documents soon, still interested in FA24'
 'Student is looking to defer to the SP25 or FA25 term'
 'Student has the needed information, does not need assistance, and plans to enroll soon'
 'Student is having trouble contacting their acade

In [20]:
unique_dates = df['Received_At'].unique()
print(unique_dates)

<DatetimeArray>
['2023-04-28 12:15:19', '2023-04-28 13:04:05', '2023-05-01 11:14:11',
 '2023-05-01 11:16:09', '2023-05-01 11:18:02', '2023-05-01 11:19:05',
 '2023-05-01 11:21:43', '2023-05-01 11:26:12', '2023-05-01 11:29:06',
 '2023-05-01 11:30:00',
 ...
 '2024-12-24 16:16:34', '2024-12-24 16:18:03', '2024-12-24 16:20:05',
 '2024-12-24 16:21:43', '2024-12-24 16:24:35', '2024-12-24 16:27:14',
 '2024-12-24 16:30:13', '2024-12-24 16:32:12', '2024-12-24 16:34:06',
 '2024-12-24 16:36:17']
Length: 33045, dtype: datetime64[ns]


Verifying Dataset Overall 

In [21]:
# Display missing values per column
print("=== Missing Values per Column ===")
print(df.isnull().sum())

# Display number of duplicate rows
print("\n=== Duplicate Rows ===")
print(df.duplicated().sum())

# Display data types of all columns
print("\n=== Data Types of Columns ===")
print(df.dtypes)

print ("\n")
# Checking Total Rows & Columns
print(f"Total Rows: {df.shape[0]}")
print(f"Total Columns: {df.shape[1]}")

=== Missing Values per Column ===
Reference_ID           0
Received_At            0
University             0
Caller_Name            0
Outcome_1              0
Remark                 0
Campaign_ID            0
Escalation_Required    0
dtype: int64

=== Duplicate Rows ===
0

=== Data Types of Columns ===
Reference_ID                   object
Received_At            datetime64[ns]
University                     object
Caller_Name                    object
Outcome_1                      object
Remark                         object
Campaign_ID                    object
Escalation_Required            object
dtype: object


Total Rows: 33118
Total Columns: 8


Viewing Dataset after Data Cleaning

In [22]:
df.head(10)

,Reference_ID,Received_At,University,Caller_Name,Outcome_1,Remark,Campaign_ID,Escalation_Required
0,12345,2023-04-28 12:15:19,Illinois Institute of Technology,Shailja,Connected,No Remark,IANF23,No
1,12345,2023-04-28 13:04:05,Illinois Institute of Technology,Shailja,Reschedule,No Remark,IANF23,No
2,12345,2023-05-01 11:14:11,Illinois Institute of Technology,Shailja,Connected,No Remark,IANF23,No
3,347397,2023-05-01 11:16:09,Illinois Institute of Technology,Isha,Not connected,No Remark,IANF23,No
4,347397,2023-05-01 11:18:02,Illinois Institute of Technology,Isha,Connected,No Remark,IANF23,No
5,358065,2023-05-01 11:19:05,Illinois Institute of Technology,Isha,Not connected,No Remark,IANF23,No
6,351333,2023-05-01 11:21:43,Illinois Institute of Technology,Isha,Not connected,No Remark,IANF23,No
7,346435,2023-05-01 11:26:12,Illinois Institute of Technology,Isha,Will Submit the docx,within few days,IANF23,No
8,355959,2023-05-01 11:29:06,Illinois Institute of Technology,Isha,Completed application,No Remark,IANF23,No
9,351520,2023-05-01 11:30:00,Illinois Institute of Technology,Shailja,Not connected,No Remark,IANF23,No


Exporting Cleaned Dataset

In [23]:
df.to_csv("Cleaned_OutreachData.csv", index=False)

# Finally "OutreachData.csv" Dataset is Cleaned !